In [137]:
from stlayout.treemap import TreeMap
from stlayout.hierarchyUtils import clean_idle_nodes, set_nodes_height
from fa2 import ForceAtlas2
import networkx as nx
import numpy as np
import json

In [138]:
filename = 'celegansneural-layout.json-flat.json'

## Load

In [139]:
with open(f'./result/{filename}') as file:
    data = json.load(file)

## Transformation Functions

In [140]:
def normalize(data):

    for node in data['nodes']:
        pos = np.array([node['pos3D'][0], node['pos3D'][2], node['pos3D'][1]])
        pos *= 2
        pos += -1

        node['_position3D'] = {
            'x': pos[0],
            'y': pos[1],
            'z': pos[2]
        }

In [141]:
def centralize_groups(data):
    groups = np.unique(np.array([node['ancIdx'] for node in data['nodes'] if node['childIdx'] == []]))
    
    for group in groups:
        positions = [node['_position3D'] for node in data['nodes'] if node['ancIdx'] == group and node['childIdx'] == []]
        cx = np.mean([pos['x'] for pos in positions])
        maxx = np.max([pos['x'] for pos in positions])
        minx = np.min([pos['x'] for pos in positions])
        cz = np.mean([pos['z'] for pos in positions])
        maxz = np.max([pos['z'] for pos in positions])
        minz = np.min([pos['z'] for pos in positions])

        for position in positions:
            position['x'] -= cx
            position['x'] /= (maxx - minx)
            position['z'] -= cz + 0.15 # one meter additional offset
            position['z'] /= (maxz - minz)

In [142]:
normalize(data)
centralize_groups(data)

## Save

In [143]:
with open(f'./result/{filename}', 'w') as file:
    json.dump(data, file)